# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Data

We will be using articles scraped from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [1]:
import pandas as pd

In [4]:
npr = pd.read_csv('../../data/npr.csv')

In [5]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [7]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [8]:
dtm = tfidf.fit_transform(npr['Article'])

In [9]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## NMF

In [10]:
from sklearn.decomposition import NMF

In [11]:
nmf_model = NMF(n_components=7,random_state=42)

In [12]:
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

c:\users\amardeep\appdata\local\programs\python\python37\lib\site-packages\sklearn\decomposition\_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


NMF(n_components=7, random_state=42)

## Displaying Topics

In [22]:
len(tfidf.get_feature_names()), tfidf.get_feature_names()[30043]

(54777, 'marais')

In [23]:
import random

In [24]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

hails
schoolwide
bantered
writing
miriam
maurice
enfants
festivities
popole
close


In [25]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names()[random_word_id])

swarthmore
reanalyzed
setter
unions
armas
educational
peeler
mikhail
demining
unobtrusive


In [26]:
len(nmf_model.components_)

7

In [27]:
nmf_model.components_

array([[0.00000000e+00, 2.49950821e-01, 0.00000000e+00, ...,
        1.70313822e-03, 2.37544362e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.22048918e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 3.12379960e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.89723338e-03, 0.00000000e+00, 1.50186440e-03, ...,
        7.06428924e-04, 5.85500542e-04, 6.89536542e-04],
       [4.01763234e-03, 5.31643833e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [28]:
len(nmf_model.components_[0])

54777

In [29]:
single_topic = nmf_model.components_[0]

In [30]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([    0, 27208, 27206, ..., 36283, 54692, 42993], dtype=int64)

In [31]:
# Word least representative of this topic
single_topic[18302]

0.0

In [32]:
# Word most representative of this topic
single_topic[42993]

2.005055165418592

In [33]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([14441, 36310, 53989, 52615, 47218, 53152, 19307, 36283, 54692,
       42993], dtype=int64)

In [34]:
top_word_indices = single_topic.argsort()[-10:]

In [35]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

disease
percent
women
virus
study
water
food
people
zika
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [36]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPIC #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPIC #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']


THE TOP 15 WORDS FOR TOPIC #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPIC #5
['love', 've', 'don', 'al

In [49]:
obtained_topics = {0: 'Virus & Disease Research', 1: 'Presidential Election' , 2: 'US healthcare', 3: 'Defence & Security', 
                   4: 'Election News', 5: 'Love & Music', 6:'Child Care And Education'}

### Attaching Discovered Topic Labels to Original Articles

In [50]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [51]:
dtm.shape

(11992, 54777)

In [52]:
len(npr)

11992

In [53]:
topic_results = nmf_model.transform(dtm)

In [54]:
topic_results.shape

(11992, 7)

In [55]:
topic_results[0]

array([0.        , 0.12075603, 0.00140297, 0.05919954, 0.01518909,
       0.        , 0.        ])

In [56]:
topic_results[0].round(2)

array([0.  , 0.12, 0.  , 0.06, 0.02, 0.  , 0.  ])

In [57]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [58]:
npr.head()

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6


In [46]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 0, 4, 3], dtype=int64)

In [59]:
npr['Topic'] = topic_results.argmax(axis=1)
npr['Topic'] = npr['Topic'].map(obtained_topics)

In [60]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",Presidential Election
1,Donald Trump has used Twitter — his prefe...,Presidential Election
2,Donald Trump is unabashedly praising Russian...,Presidential Election
3,"Updated at 2:50 p. m. ET, Russian President Vl...",Defence & Security
4,"From photography, illustration and video, to d...",Child Care And Education
5,I did not want to join yoga class. I hated tho...,Love & Music
6,With a who has publicly supported the debunk...,Virus & Disease Research
7,"I was standing by the airport exit, debating w...",Virus & Disease Research
8,"If movies were trying to be more realistic, pe...",Virus & Disease Research
9,"Eighteen years ago, on New Year’s Eve, David F...",Love & Music


## Great work!